In [12]:
import torch
from utils.fda import FDA_source_to_target
from utils.aug import ExtCompose, ExtToTensor, ExtRandomHorizontalFlip , ExtScale , ExtRandomCrop, ExtGaussianBlur, ExtColorJitter, ExtRandomCompose, ExtResize, ExtNormalize
from datasets.cityscapes import CityScapes
from datasets.GTA5 import GTA5
from torch.utils.data import DataLoader
from PIL import Image
import numpy as np

## FDA TEST

In [2]:
beta = 0.05
h = 512
w = 1024
b = (  np.floor(np.amin((h,w))*beta)  ).astype(int)
b

25

In [3]:
standard_transformations = ExtCompose([ExtScale(0.5,interpolation=Image.Resampling.BILINEAR), ExtToTensor()])
transformations = ExtCompose([ExtResize((512,1024)), ExtToTensor()])
target_transformations = standard_transformations

train_dataset = GTA5(root='dataset',transforms=transformations)
target_dataset_train = CityScapes(split='train',transforms=target_transformations)

source_dataloader_train = DataLoader(train_dataset,
                batch_size=1,
                shuffle=False,
                num_workers=1,
                pin_memory=False,
                drop_last=True)
target_dataloader_train = DataLoader(target_dataset_train,
                batch_size=1,
                shuffle=False,
                num_workers=1,
                pin_memory=False,
                drop_last=True)

for i, (source_data, target_data) in enumerate(zip(source_dataloader_train,target_dataloader_train)):
  
    if i > 10:
        break

    source_data,source_label = source_data
    target_data,_ = target_data
    source_data = source_data.cuda()
    source_label = source_label.long().cuda()
    target_data = target_data.cuda()

    t_source_data = FDA_source_to_target(source_data, target_data, beta=beta)

    # Visualize the image
    #Image.fromarray(target_data.detach().cpu().numpy().astype('uint8')[0].transpose(1, 2, 0)).show()
    #Image.fromarray(source_data.detach().cpu().numpy().astype('uint8')[0].transpose(1, 2, 0)).show()
    Image.fromarray(t_source_data.detach().cpu().numpy().astype('uint8')[0].transpose(1, 2, 0)).show()

root:  dataset


In [23]:
GTA_MEAN = torch.tensor([105.8735, 109.0072, 101.7702])
GTA_STD = torch.tensor([56.2629, 55.8382, 55.4247])

In [25]:
standard_transformations = ExtCompose([ExtResize((512,1024)), ExtToTensor()])
transformations = ExtCompose([ExtResize((512,1024)), ExtToTensor(), ExtNormalize(mean=GTA_MEAN,std=GTA_STD)])
target_transformations = standard_transformations

train_dataset = GTA5(root='dataset',transforms=transformations)
target_dataset_train = CityScapes(split='train',transforms=transformations)

source_dataloader_train = DataLoader(train_dataset,
                batch_size=1,
                shuffle=False,
                num_workers=6,
                pin_memory=False,
                drop_last=True)
target_dataloader_train = DataLoader(target_dataset_train,
                batch_size=1,
                shuffle=False,
                num_workers=6,
                pin_memory=False,
                drop_last=True)
mean = 0.
std = 0.
K = 1000
print(len(source_dataloader_train))
for i , (data , labels) in enumerate(source_dataloader_train):
    mean += data.mean(keepdim=True,dim=(0,2,3))
    std += data.std(keepdim=True, dim=(0,2,3))
    if i % 100 == 0:
        print(i)
for i , (data , labels) in enumerate(target_dataloader_train):
    mean += data.mean(keepdim=True,dim=(0,2,3))
    std += data.std(keepdim=True, dim=(0,2,3))
    if i % 100 == 0:
        print(i)
mean /= (len(source_dataloader_train) + len(target_dataloader_train))
std /= (len(source_dataloader_train) + len(target_dataloader_train))
print(mean.view(-1))
print(std.view(-1))

root:  dataset
2000
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
tensor([ 2.4673e-06, -2.8932e-07,  2.0480e-06])
tensor([1.0000, 1.0000, 1.0000])
